In [23]:
import re
import spacy
import pdfplumber

In [24]:
!pip install pdfplumber

In [25]:
nlp=spacy.load("en_core_web_sm")

In [32]:
def extract_invoice_data(file_path):
    with pdfplumber.open(file_path)as pdf:
        text="\n".join([page.extract_text() for page in  pdf.pages if page.extract_text()])

        x=nlp(text)
        vendor=None
        for ent in x.ents:
            if ent.label =="ORG":
                vendor=ent.txt
                break
        date=re.search(r'(\d{1,2}[/-]\d{1,2}[\-]\d{2,4})',text)
        in_date = date.group(0) if date else "Not Found"

        amt=re.search(r'Total(?:Amount)?[:\s]*\$?([0-9,.]+)', text)
        total=amt.group(1) if amt else "Not Found"

        Pan=re.search(r'\b[A-Z]{5}[0-9]{4}[A-Z]\b', text)
        PAN=Pan.group(0) if amt else "Not Found"

        
        return {
                "Company":vendor,
                "Invoice date": in_date,
                "Total Amount": total,
                "Pan":PAN
        }
file_path="inv.pdf"
invoice_info=extract_invoice_data(file_path)
print(invoice_info)
        

{'Company': None, 'Invoice date': '10-11-2024', 'Total Amount': '2', 'Pan': 'AAICB4842N'}
